In [ ]:
# ===================================================================
# k-NN PRESERVATION ANALYSIS (HSCC SLIDE 3)
# ===================================================================
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr, spearmanr
import torch
import scanpy as sc 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import utils_et as uet

from datetime import datetime

print("\n" + "="*70)
print("k-NN PRESERVATION ANALYSIS - HSCC SLIDE 3")
print("="*70)

# Load your inference results
# Adjust the timestamp to match your actual saved file
output_dir = "/home/ehtesamul/sc_st/model/gems_hscc_output_anchored"  # UPDATE THIS PATH
timestamp = "20260119_181640"  # UPDATE THIS, e.g., "20251208_143052"

# Load processed results
results_path = f"{output_dir}/st3_inference_processed_{timestamp}.pt"
results = torch.load(results_path, map_location='cpu', weights_only=False)

# Extract predicted coordinates and EDM
coords_pred = results['coords_canon']  # Shape: (n_spots, 2)
D_edm_pred = results['D_edm']  # Shape: (n_spots, n_spots)

# Load ground truth from stadata3
stadata3 = sc.read_h5ad('/home/ehtesamul/sc_st/data/cSCC/processed/stP2rep3.h5ad')
gt_coords = stadata3.obsm['spatial']  # Shape: (n_spots, 2)

# Convert to tensor and canonicalize (single slide → all slide_ids = 0)
gt_coords_tensor = torch.tensor(gt_coords, dtype=torch.float32)
slide_ids = torch.zeros(gt_coords_tensor.shape[0], dtype=torch.long)
gt_coords_tensor, st_mu, st_scale = uet.canonicalize_st_coords_per_slide(
    gt_coords_tensor, slide_ids
)

# Convert back to numpy for downstream analysis
gt_coords = gt_coords_tensor.cpu().numpy()

n_spots = gt_coords.shape[0]
print(f"Number of spots: {n_spots}")
print(f"Ground truth coords: {gt_coords.shape}")
print(f"Predicted coords: {coords_pred.shape}")

# ===================================================================
# COMPUTE k-NN PRESERVATION
# ===================================================================

def compute_knn_preservation(coords_gt, coords_pred, k=10):
    """
    Compute k-nearest neighbor preservation.
    Returns average number of preserved neighbors per spot.
    
    Parameters
    ----------
    coords_gt : ndarray, shape (n, 2)
        Ground truth coordinates
    coords_pred : ndarray, shape (n, 2)  
        Predicted coordinates
    k : int
        Number of neighbors
        
    Returns
    -------
    mean_overlap : float
        Mean number of preserved neighbors
    overlaps : list
        Per-spot overlap counts
    """
    n = coords_gt.shape[0]
    
    # Build k-NN for ground truth
    nbrs_gt = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(coords_gt)
    _, indices_gt = nbrs_gt.kneighbors(coords_gt)
    indices_gt = indices_gt[:, 1:]  # Remove self
    
    # Build k-NN for predicted
    nbrs_pred = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(coords_pred)
    _, indices_pred = nbrs_pred.kneighbors(coords_pred)
    indices_pred = indices_pred[:, 1:]  # Remove self
    
    # Compute overlap for each spot
    overlaps = np.array([
        len(set(indices_gt[i]) & set(indices_pred[i]))
        for i in range(n)
    ])
    
    return overlaps.mean(), overlaps

# Compute for k=10 and k=20
knn_k10, overlaps_k10 = compute_knn_preservation(gt_coords, coords_pred, k=10)
knn_k20, overlaps_k20 = compute_knn_preservation(gt_coords, coords_pred, k=20)

print(f"\nk-NN Preservation Results:")
print(f"  k=10: {knn_k10:.2f} / 10  ({knn_k10/10*100:.1f}% neighbors preserved)")
print(f"  k=20: {knn_k20:.2f} / 20  ({knn_k20/20*100:.1f}% neighbors preserved)")

# ===================================================================
# COMPUTE EDM CORRELATIONS FOR COMPARISON
# ===================================================================

# Compute ground truth EDM
gt_edm = squareform(pdist(gt_coords, 'euclidean'))

# Extract upper triangle distances
triu_indices = np.triu_indices(n_spots, k=1)
gt_distances = gt_edm[triu_indices]
pred_distances = D_edm_pred[triu_indices]

# Scale alignment
scale = np.median(gt_distances) / np.median(pred_distances)
pred_distances_scaled = pred_distances * scale

# Correlations
pearson_corr, _ = pearsonr(gt_distances, pred_distances_scaled)
spearman_corr, _ = spearmanr(gt_distances, pred_distances_scaled)

print(f"\nComparison:")
print(f"  EDM Pearson:  {pearson_corr:.4f}  (global distances)")
print(f"  EDM Spearman: {spearman_corr:.4f}  (global distances)")
print(f"  k-NN@10:      {knn_k10/10:.4f}  (local neighborhoods)")
print(f"  k-NN@20:      {knn_k20/20:.4f}  (local neighborhoods)")

print("\n" + "="*70)
print("INTERPRETATION")
print("="*70)

if knn_k10/10 < 0.3:
    print("\n⚠️ SEVERE local scrambling detected!")
    print("   < 30% of neighbors preserved - local structure heavily disrupted")
elif knn_k10/10 < 0.5:
    print("\n⚠️ MODERATE local scrambling detected")
    print("   30-50% of neighbors preserved - significant local disruption")
elif knn_k10/10 < 0.7:
    print("\n✓ MILD local scrambling")
    print("   50-70% of neighbors preserved - some local structure retained")
else:
    print("\n✓✓ GOOD local preservation")
    print("   > 70% of neighbors preserved - local structure mostly intact")

if pearson_corr > 0.6 and knn_k10/10 < 0.4:
    print("\n🔍 DIAGNOSIS: High EDM correlation but low k-NN preservation")
    print("   → Global geometry preserved, but specific neighbor relationships lost")
    print("   → Consistent with context-dependent coordinate generation")

# ===================================================================
# DISTANCE-DEPENDENT ERROR ANALYSIS
# ===================================================================

print("\n" + "="*70)
print("DISTANCE-DEPENDENT ERROR ANALYSIS")
print("="*70)

n_bins = 20
bins = np.linspace(0, np.percentile(gt_distances, 95), n_bins + 1)
bin_idx = np.digitize(gt_distances, bins)

print(f"\n{'Bin':<5} {'Mean Dist':<12} {'RMSE':<12} {'Rel Error':<12} {'N Points':<10}")
print("-" * 60)

for b in range(1, n_bins + 1):
    mask = bin_idx == b
    if mask.sum() > 100:
        gt_bin = gt_distances[mask]
        pred_bin = pred_distances_scaled[mask]
        
        rmse = np.sqrt(((gt_bin - pred_bin) ** 2).mean())
        mean_dist = gt_bin.mean()
        rel_err = rmse / mean_dist
        n_points = mask.sum()
        
        print(f"{b:<5} {mean_dist:<12.4f} {rmse:<12.4f} {rel_err:<12.4f} {n_points:<10}")

# ===================================================================
# VISUALIZATION: k-NN PRESERVATION HEATMAP
# ===================================================================

print("\n" + "="*70)
print("VISUALIZING k-NN PRESERVATION")
print("="*70)

fig, axes = plt.subplots(1, 3, figsize=(24, 7))

# Plot 1: Ground truth coordinates
ax = axes[0]
if 'celltype' in stadata3.obs.columns:
    cell_types = stadata3.obs['celltype']
    unique_types = cell_types.unique()
    cmap = plt.cm.tab10 if len(unique_types) <= 10 else plt.cm.tab20
    
    for i, ct in enumerate(unique_types):
        mask = cell_types == ct
        ax.scatter(gt_coords[mask, 0], gt_coords[mask, 1],
                  c=[cmap(i / len(unique_types))], label=ct, s=20, alpha=0.7)
else:
    ax.scatter(gt_coords[:, 0], gt_coords[:, 1], s=20, alpha=0.7, c='blue')

ax.set_title('Ground Truth (Slide 3)', fontsize=16, weight='bold')
ax.set_xlabel('X', fontsize=14)
ax.set_ylabel('Y', fontsize=14)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

# Plot 2: Predicted coordinates
ax = axes[1]
if 'celltype' in stadata3.obs.columns:
    for i, ct in enumerate(unique_types):
        mask = cell_types == ct
        ax.scatter(coords_pred[mask, 0], coords_pred[mask, 1],
                  c=[cmap(i / len(unique_types))], label=ct, s=20, alpha=0.7)
else:
    ax.scatter(coords_pred[:, 0], coords_pred[:, 1], s=20, alpha=0.7, c='red')

ax.set_title('GEMS Predicted', fontsize=16, weight='bold')
ax.set_xlabel('X', fontsize=14)
ax.set_ylabel('Y', fontsize=14)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

# Plot 3: k-NN preservation per spot
ax = axes[2]
scatter = ax.scatter(coords_pred[:, 0], coords_pred[:, 1],
                    c=overlaps_k10, cmap='RdYlGn', s=20, vmin=0, vmax=10, alpha=0.8)
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Neighbors Preserved (k=10)', fontsize=12)
ax.set_title(f'k-NN@10 Preservation\nMean: {knn_k10:.2f}/10 ({knn_k10/10*100:.1f}%)', 
            fontsize=16, weight='bold')
ax.set_xlabel('X', fontsize=14)
ax.set_ylabel('Y', fontsize=14)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

if 'celltype' in stadata3.obs.columns and len(unique_types) <= 15:
    handles, labels = axes[1].get_legend_handles_labels()
    fig.legend(handles, labels, loc='center left', bbox_to_anchor=(1.0, 0.5),
              fontsize=10, title='Cell Type', title_fontsize=12)

plt.tight_layout(rect=[0, 0, 0.95, 1])
knn_plot_path = f"{output_dir}/hscc_knn_preservation_{timestamp}.png"
# plt.savefig(knn_plot_path, dpi=300, bbox_inches='tight')
print(f"\n✓ Saved k-NN preservation plot: {knn_plot_path}")
plt.show()

print("\n" + "="*70)
print("k-NN PRESERVATION ANALYSIS COMPLETE")
print("="*70)

In [ ]:
import torch
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import sys
import matplotlib.pyplot as plt
sys.path.insert(0, '/home/ehtesamul/sc_st/model')

from core_models_et_p3 import GEMSModel, infer_anchor_train_from_checkpoint
from core_models_et_p1 import STSetDataset, SCSetDataset
import utils_et as uet

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ===================================================================
# 1. LOAD HSCC DATA
# ===================================================================
print("Loading HSCC data...")

# Load SC data
scadata = sc.read_h5ad('/home/ehtesamul/sc_st/data/cSCC/processed/scP2.h5ad')

# Load 3 ST slides (first 2 for training, 3rd for inference)
stadata1 = sc.read_h5ad('/home/ehtesamul/sc_st/data/cSCC/processed/stP2.h5ad')
stadata2 = sc.read_h5ad('/home/ehtesamul/sc_st/data/cSCC/processed/stP2rep2.h5ad')
stadata3 = sc.read_h5ad('/home/ehtesamul/sc_st/data/cSCC/processed/stP2rep3.h5ad')  # For inference

# Normalize
print("Normalizing data...")
for adata in [scadata, stadata1, stadata2, stadata3]:
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)

# Create cell types for SC data
print("Creating cell types...")
scadata.obs['celltype'] = scadata.obs['level1_celltype'].astype(str)
scadata.obs.loc[scadata.obs['level1_celltype']=='CLEC9A', 'celltype'] = 'DC'
scadata.obs.loc[scadata.obs['level1_celltype']=='CD1C', 'celltype'] = 'DC'
scadata.obs.loc[scadata.obs['level1_celltype']=='ASDC', 'celltype'] = 'DC'
scadata.obs.loc[scadata.obs['level1_celltype']=='PDC', 'celltype'] = 'PDC'
scadata.obs.loc[scadata.obs['level1_celltype']=='MDSC', 'celltype'] = 'DC'
scadata.obs.loc[scadata.obs['level1_celltype']=='LC', 'celltype'] = 'DC'
scadata.obs.loc[scadata.obs['level1_celltype']=='Mac', 'celltype'] = 'Myeloid cell'
scadata.obs.loc[scadata.obs['level1_celltype']=='Tcell', 'celltype'] = 'T cell'
scadata.obs.loc[scadata.obs['level2_celltype']=='TSK', 'celltype'] = 'TSK'
scadata.obs.loc[scadata.obs['level2_celltype'].isin(['Tumor_KC_Basal', 'Tumor_KC_Diff', 'Tumor_KC_Cyc']), 'celltype'] = 'NonTSK'

# Get common genes
common = sorted(list(set(scadata.var_names) & set(stadata1.var_names) & 
                     set(stadata2.var_names) & set(stadata3.var_names)))
n_genes = len(common)

print(f"✓ SC data: {scadata.shape[0]} cells × {scadata.shape[1]} genes")
print(f"✓ ST slide 1: {stadata1.shape[0]} spots × {stadata1.shape[1]} genes")
print(f"✓ ST slide 2: {stadata2.shape[0]} spots × {stadata2.shape[1]} genes")
print(f"✓ ST slide 3 (inference): {stadata3.shape[0]} spots × {stadata3.shape[1]} genes")
print(f"✓ Common genes: {n_genes}")

# Extract tensors for training slides
X_st1 = stadata1[:, common].X
X_st2 = stadata2[:, common].X
if hasattr(X_st1, "toarray"): X_st1 = X_st1.toarray()
if hasattr(X_st2, "toarray"): X_st2 = X_st2.toarray()

# Combine training ST data
st_expr_combined = torch.tensor(np.vstack([X_st1, X_st2]), dtype=torch.float32, device=device)

# ST coordinates from 2 training slides
st_coords1 = stadata1.obsm['spatial']
st_coords2 = stadata2.obsm['spatial']
st_coords_raw = torch.tensor(np.vstack([st_coords1, st_coords2]), dtype=torch.float32, device=device)

# Slide IDs (0 for slide1, 1 for slide2)
slide_ids = torch.tensor(
    np.concatenate([
        np.zeros(X_st1.shape[0], dtype=int),
        np.ones(X_st2.shape[0], dtype=int)
    ]),
    dtype=torch.long, device=device
)

# Per-slide canonicalization
st_coords, st_mu, st_scale = uet.canonicalize_st_coords_per_slide(st_coords_raw, slide_ids)

print(f"✓ Combined ST data: {st_expr_combined.shape[0]} spots")
print(f"✓ Slide IDs: slide 0 ({(slide_ids==0).sum()} spots), slide 1 ({(slide_ids==1).sum()} spots)")
print(f"✓ Coordinates canonicalized: scale={st_scale.cpu().numpy()}")

# ===================================================================
# 2. LOAD TRAINED MODEL
# ===================================================================
print("\n" + "="*70)
print("LOADING TRAINED MODEL")
print("="*70)

output_dir = "/home/ehtesamul/sc_st/model/gems_hscc_output_anchored"
checkpoint_path = f"{output_dir}/phase1_st_checkpoint.pt"

# Auto-detect anchor mode from checkpoint
checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
base_h_dim = 128  # Last dimension of n_embedding
anchor_train_detected = infer_anchor_train_from_checkpoint(checkpoint, base_h_dim)
print(f"✓ Detected anchor_train={anchor_train_detected}")

model = GEMSModel(
    n_genes=n_genes,
    n_embedding=[512, 256, 128],
    D_latent=32,
    c_dim=256,
    n_heads=4,
    isab_m=128,
    dist_bins=24,
    device=device,
    anchor_train=anchor_train_detected,
)

# Load weights
model.encoder.load_state_dict(checkpoint['encoder'])
model.context_encoder.load_state_dict(checkpoint['context_encoder'])
model.generator.load_state_dict(checkpoint['generator'])
model.score_net.load_state_dict(checkpoint['score_net'])

# Restore EDM parameters
if 'sigma_data' in checkpoint:
    model.sigma_data = checkpoint['sigma_data']
if 'sigma_min' in checkpoint:
    model.sigma_min = checkpoint['sigma_min']
if 'sigma_max' in checkpoint:
    model.sigma_max = checkpoint['sigma_max']

model.encoder.eval()
model.context_encoder.eval()
model.generator.eval()
model.score_net.eval()

print(f"✓ Model loaded from {checkpoint_path}")
print(f"✓ anchor_train={model.anchor_train}")
print(f"✓ sigma_data={getattr(model, 'sigma_data', 'N/A')}")

# ===================================================================
# 3. RUN STAGE B ON TRAINING SLIDES
# ===================================================================
print("\n" + "="*70)
print("RUNNING STAGE B")
print("="*70)

# Separate coordinates for each training slide
st1_coords_canon = st_coords[slide_ids == 0]
st1_expr = st_expr_combined[slide_ids == 0]
st2_coords_canon = st_coords[slide_ids == 1]
st2_expr = st_expr_combined[slide_ids == 1]

slides_dict = {
    0: (st1_coords_canon, st1_expr),
    1: (st2_coords_canon, st2_expr)
}

model.train_stageB(
    slides=slides_dict,
    outdir=f"{output_dir}/stageB_targets"
)
print("✓ Stage B complete")

# ===================================================================
# 4. CREATE MINI-SET DATASETS (FROM TRAINING SLIDES)
# ===================================================================
print("\n" + "="*70)
print("CREATING MINI-SET DATASETS")
print("="*70)

st_gene_expr_dict_cpu = {
    0: st1_expr.cpu(),
    1: st2_expr.cpu()
}

st_dataset = STSetDataset(
    targets_dict=model.targets_dict,
    encoder=model.encoder,
    st_gene_expr_dict=st_gene_expr_dict_cpu,
    n_min=128,
    n_max=384,
    D_latent=model.D_latent,
    num_samples=20,  # Sample from both slides
    knn_k=12,
    device=device,
    landmarks_L=0,
    pool_mult=2.0,
    stochastic_tau=1.0,
)

print(f"✓ ST dataset: {len(st_dataset)} mini-sets (from 2 training slides)")

# ===================================================================
# 5. SAMPLE MINI-SETS
# ===================================================================
print("\nSampling mini-sets...")

st_minisets = []
with torch.no_grad():
    for i in range(len(st_dataset)):
        st_minisets.append(st_dataset[i])

print(f"✓ Sampled {len(st_minisets)} ST mini-sets")
print(f"\nST mini-set #0 keys: {list(st_minisets[0].keys())}")
print(f"  Z_set shape: {st_minisets[0]['Z_set'].shape}")
print(f"  V_target shape: {st_minisets[0]['V_target'].shape}")
print(f"  n points: {st_minisets[0]['n']}")

# ===================================================================
# 6. PLOT MINI-SETS
# ===================================================================
print("\nPlotting mini-sets...")

n_st_plots = 6
max_cols = 3

n_rows_st = (n_st_plots + max_cols - 1) // max_cols
n_cols_st = min(n_st_plots, max_cols)

fig_st, axes_st = plt.subplots(n_rows_st, n_cols_st, 
                                figsize=(6.5*n_cols_st, 6*n_rows_st))
axes_st = np.atleast_2d(axes_st).reshape(n_rows_st, n_cols_st)

for i in range(n_st_plots):
    row, col = i // max_cols, i % max_cols
    ax = axes_st[row, col]
    coords = st_minisets[i]['V_target'].cpu().numpy()
    n = st_minisets[i]['n']
    ax.scatter(coords[:n, 0], coords[:n, 1], s=20, alpha=0.7)
    ax.set_title(f'ST Mini-set {i} (n={n})', fontsize=10)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

for i in range(n_st_plots, n_rows_st * n_cols_st):
    row, col = i // max_cols, i % max_cols
    axes_st[row, col].axis('off')

plt.tight_layout()
plt.show()

print("\n✓ MINI-SETS READY!")


In [ ]:
# ===================================================================
# CREATE SC MINISETS FOR INFERENCE SLIDE (stadata3)
# ===================================================================
print("="*70)
print("PREPARING INFERENCE DATA (SLIDE 3)")
print("="*70)

# Extract inference slide expression
X_st3 = stadata3[:, common].X
if hasattr(X_st3, "toarray"):
    X_st3 = X_st3.toarray()
st3_expr = torch.tensor(X_st3, dtype=torch.float32, device=device)

print(f"✓ Inference slide (ST3): {st3_expr.shape[0]} spots × {st3_expr.shape[1]} genes")

# Check if GT coordinates exist in obs columns (new_x, new_y)
if 'new_x' in stadata3.obs.columns and 'new_y' in stadata3.obs.columns:
    print("✓ Ground truth coordinates available in obs (new_x, new_y)")
    gt_coords_available = True
    gt_coords_raw_st3 = stadata3.obs[['new_x', 'new_y']].values
    print(f"✓ GT coords shape: {gt_coords_raw_st3.shape}")
    print(f"✓ GT coords range: X=[{gt_coords_raw_st3[:,0].min():.3f}, {gt_coords_raw_st3[:,0].max():.3f}], "
          f"Y=[{gt_coords_raw_st3[:,1].min():.3f}, {gt_coords_raw_st3[:,1].max():.3f}]")
elif 'spatial_gt' in stadata3.obsm:
    print("✓ Ground truth coordinates available in obsm['spatial_gt']")
    gt_coords_available = True
    gt_coords_raw_st3 = stadata3.obsm['spatial_gt']
    print(f"✓ GT coords shape: {gt_coords_raw_st3.shape}")
else:
    print("⚠️ No ground truth coordinates for slide 3")
    gt_coords_available = False
    gt_coords_raw_st3 = None

print("\n✓ Inference data ready!")


In [ ]:
import torch
import numpy as np
from datetime import datetime
import sys
import os
import scanpy as sc
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.insert(0, '/home/ehtesamul/sc_st/model')
from core_models_et_p3 import GEMSModel, infer_anchor_train_from_checkpoint
import utils_et as uet

# ===================================================================
# PATHS AND CONFIG
# ===================================================================
output_dir = "/home/ehtesamul/sc_st/model/gems_hscc_output_anchored"
checkpoint_path = f"{output_dir}/phase1_st_checkpoint.pt"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("="*70)
print("LOADING DATA AND MODEL FOR INFERENCE")
print("="*70)

# ===================================================================
# LOAD AND NORMALIZE GT COORDS (IF AVAILABLE)
# ===================================================================
gt_coords_norm = None
if gt_coords_available and gt_coords_raw_st3 is not None:
    gt_coords_tensor = torch.tensor(gt_coords_raw_st3, dtype=torch.float32, device=device)
    slide_ids_gt = torch.zeros(gt_coords_tensor.shape[0], dtype=torch.long, device=device)
    
    gt_coords_norm, gt_mu, gt_scale = uet.canonicalize_st_coords_per_slide(
        gt_coords_tensor, slide_ids_gt
    )
    
    gt_coords_np = gt_coords_norm.cpu().numpy()
    print(f"✓ GT coords normalized: scale={gt_scale[0].item():.4f}")
    print(f"✓ GT coords RMS: {gt_coords_norm.pow(2).mean().sqrt().item():.4f}")
    print(f"✓ GT coords range: X=[{gt_coords_np[:,0].min():.3f}, {gt_coords_np[:,0].max():.3f}], "
          f"Y=[{gt_coords_np[:,1].min():.3f}, {gt_coords_np[:,1].max():.3f}]")
else:
    print("⚠️ No GT coords available - debug_knn will be disabled")

# ===================================================================
# MODEL ALREADY LOADED FROM CELL 1, BUT RELOAD CHECKPOINT PARAMS
# ===================================================================
checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

if 'sigma_data' in checkpoint:
    model.sigma_data = checkpoint['sigma_data']
if 'sigma_min' in checkpoint:
    model.sigma_min = checkpoint['sigma_min']
if 'sigma_max' in checkpoint:
    model.sigma_max = checkpoint['sigma_max']

print(f"✓ Model ready for inference")
print(f"  sigma_data={getattr(model, 'sigma_data', 'N/A')}")
print(f"  sigma_min={getattr(model, 'sigma_min', 'N/A')}")
print(f"  sigma_max={getattr(model, 'sigma_max', 'N/A')}")

# ===================================================================
# COMPUTE CORAL TRANSFORMATION
# ===================================================================
print("\n" + "="*70)
print("COMPUTING CORAL TRANSFORMATION")
print("="*70)

# Prepare ST gene expression dict from training slides
st_gene_expr_dict = {
    0: st1_expr.cpu(),
    1: st2_expr.cpu()
}

# # Compute ST context distribution
# print("--- Computing ST context distribution (from 2 training slides) ---")
# model.compute_coral_params_from_st(
#     st_gene_expr_dict=st_gene_expr_dict,
#     n_samples=2000,
#     n_min=96,
#     n_max=384,
# )

# # Build CORAL transform using inference slide expression
# print("--- Building CORAL transformation for inference slide ---")
# model.build_coral_transform(
#     sc_gene_expr=st3_expr,  # Use inference slide as "SC" data
#     n_samples=2000,
#     n_min=96,
#     n_max=384,
#     shrink=0.01,
#     eps=1e-5,
# )

# print("✓ CORAL transformation ready!")

# print("\n" + "="*70)

# ===================================================================
# RUN INFERENCE (PATCHWISE)
# ===================================================================
print("\n" + "="*70)
print("RUNNING INFERENCE ON SLIDE 3")
print("="*70)

n_cells = st3_expr.shape[0]

with torch.no_grad():
    results = model.infer_sc_patchwise(
        sc_gene_expr=st3_expr,
        n_timesteps_sample=500,
        return_coords=True,
        patch_size=638,
        coverage_per_cell=1.0,
        n_align_iters=1,
        eta=0.0,
        guidance_scale=2.0,
        gt_coords=gt_coords_norm,
        debug_knn=(gt_coords_norm is not None),  # Only if GT available
        debug_max_patches=15,
        debug_k_list=(10, 20),
        pool_mult=2.0,
        stochastic_tau=0.8,
        tau_mode="adaptive_kth",
        ensure_connected=True,
        local_refine=False,
        # ========== ANCHORED INFERENCE ==========
        inference_mode="anchored",
        anchor_sampling_mode="align_vote_only",
        commit_frac=0.6,
        seq_align_dim=32,
        coldstart_diag=True
    )

D_edm_pred = results['D_edm'].cpu().numpy()
coords_pred = results['coords_canon'].cpu().numpy()
print(f"\n✓ Inference complete!")
print(f"  Predicted EDM: {D_edm_pred.shape}")
print(f"  Predicted coords: {coords_pred.shape}")


In [ ]:
# ===================================================================
# COMPUTE GROUND TRUTH EDM AND METRICS (IF GT AVAILABLE)
# ===================================================================
if gt_coords_norm is not None:
    print("\n" + "="*70)
    print("COMPUTING METRICS")
    print("="*70)
    
    gt_edm = squareform(pdist(gt_coords_np, 'euclidean'))
    print(f"✓ Ground truth EDM: {gt_edm.shape}")
    
    # Extract upper triangle distances
    triu_indices = np.triu_indices(n_cells, k=1)
    gt_distances = gt_edm[triu_indices]
    pred_distances = D_edm_pred[triu_indices]
    
    # Scale alignment
    scale = np.median(gt_distances) / np.median(pred_distances)
    pred_distances_scaled = pred_distances * scale
    
    # Correlations
    pearson_corr, _ = pearsonr(gt_distances, pred_distances_scaled)
    spearman_corr, _ = spearmanr(gt_distances, pred_distances_scaled)
    
    print(f"\nPearson Correlation:  {pearson_corr:.4f}")
    print(f"Spearman Correlation: {spearman_corr:.4f}")
    print(f"Scale factor: {scale:.4f}")
    
    # ===================================================================
    # VISUALIZATIONS
    # ===================================================================
    print("\n" + "="*70)
    print("GENERATING PLOTS")
    print("="*70)
    
    # Plot 1: Coordinate comparison
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    axes[0].scatter(gt_coords_np[:, 0], gt_coords_np[:, 1], s=5, alpha=0.6, c='blue')
    axes[0].set_title('Ground Truth Coordinates (Slide 3)', fontsize=14, weight='bold')
    axes[0].set_aspect('equal')
    
    axes[1].scatter(coords_pred[:, 0], coords_pred[:, 1], s=5, alpha=0.6, c='red')
    axes[1].set_title('Predicted Coordinates (Slide 3)', fontsize=14, weight='bold')
    axes[1].set_aspect('equal')
    
    plt.tight_layout()
    plt.show()
    
    # Plot 2: Distance scatter - Pearson and Spearman
    sample_size = min(50000, len(gt_distances))
    sample_idx = np.random.choice(len(gt_distances), sample_size, replace=False)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Pearson
    ax = axes[0]
    ax.scatter(gt_distances[sample_idx], pred_distances_scaled[sample_idx], alpha=0.2, s=5)
    ax.set_title(f'Distance Correlation (Pearson)\nρ = {pearson_corr:.4f}', fontsize=16, weight='bold')
    ax.set_xlabel('Ground Truth Distance', fontsize=12)
    ax.set_ylabel('Predicted Distance (scaled)', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.5)
    
    lims = [min(ax.get_xlim()[0], ax.get_ylim()[0]), max(ax.get_xlim()[1], ax.get_ylim()[1])]
    ax.plot(lims, lims, 'r--', alpha=0.75, label='Ideal')
    ax.set_aspect('equal')
    ax.legend()
    
    # Spearman
    ax = axes[1]
    ax.scatter(gt_distances[sample_idx], pred_distances_scaled[sample_idx], alpha=0.2, s=5)
    ax.set_title(f'Distance Correlation (Spearman)\nρ = {spearman_corr:.4f}', fontsize=16, weight='bold')
    ax.set_xlabel('Ground Truth Distance', fontsize=12)
    ax.set_ylabel('Predicted Distance (scaled)', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.5)
    
    lims = [min(ax.get_xlim()[0], ax.get_ylim()[0]), max(ax.get_xlim()[1], ax.get_ylim()[1])]
    ax.plot(lims, lims, 'r--', alpha=0.75, label='Ideal')
    ax.set_aspect('equal')
    ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Plot 3: Distance distributions
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.histplot(gt_distances, color='blue', label='Ground Truth', stat='density', bins=100, alpha=0.5, ax=ax)
    sns.histplot(pred_distances_scaled, color='red', label='Predicted (scaled)', stat='density', bins=100, alpha=0.5, ax=ax)
    ax.set_title('Distance Distribution Comparison', fontsize=16, weight='bold')
    ax.set_xlabel('Distance')
    ax.legend()
    plt.show()
    
    # ===================================================================
    # EDM HEATMAP COMPARISON
    # ===================================================================
    print("\n" + "="*70)
    print("EDM HEATMAP VISUALIZATION")
    print("="*70)
    
    def normalize_matrix(matrix):
        min_val = matrix.min()
        max_val = matrix.max()
        return (matrix - min_val) / (max_val - min_val)
    
    gt_edm_norm = normalize_matrix(gt_edm)
    pred_edm_norm = normalize_matrix(D_edm_pred)
    
    # Sample cells
    sample_size = min(500, n_cells)
    sample_indices = np.random.choice(n_cells, sample_size, replace=False)
    sample_indices = np.sort(sample_indices)
    
    print(f"\nCreating EDM heatmaps with {sample_size} sampled cells...")
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    fig.suptitle('EDM Comparison: Ground Truth vs. Predicted', fontsize=18, fontweight='bold')
    
    # Ground Truth EDM
    im1 = axes[0].imshow(gt_edm_norm[np.ix_(sample_indices, sample_indices)], cmap='viridis')
    axes[0].set_title('Ground Truth EDM (Normalized)', fontsize=14)
    axes[0].set_xlabel('Cell Index (Sampled)', fontsize=12)
    axes[0].set_ylabel('Cell Index (Sampled)', fontsize=12)
    fig.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)
    
    # Predicted EDM
    im2 = axes[1].imshow(pred_edm_norm[np.ix_(sample_indices, sample_indices)], cmap='viridis')
    axes[1].set_title('Predicted EDM (Normalized)', fontsize=14)
    axes[1].set_xlabel('Cell Index (Sampled)', fontsize=12)
    fig.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
    
    # ===================================================================
    # ANISOTROPY ANALYSIS
    # ===================================================================
    print("\n" + "="*70)
    print("EIGENVALUE ANISOTROPY ANALYSIS")
    print("="*70)
    
    def compute_anisotropy(coords):
        X = coords.astype(float)
        Xc = X - X.mean(axis=0, keepdims=True)
        cov = Xc.T @ Xc / (Xc.shape[0] - 1)
        eigvals, eigvecs = np.linalg.eigh(cov)
        eigvals = eigvals[::-1]
        lam1, lam2 = eigvals[0], eigvals[1]
        ratio = lam1 / (lam2 + 1e-12)
        return lam1, lam2, ratio
    
    # Ground truth
    lam1_gt, lam2_gt, ratio_gt = compute_anisotropy(gt_coords_np)
    print(f"\nGround Truth Coordinates:")
    print(f"  λ₁ = {lam1_gt:.4f},  λ₂ = {lam2_gt:.4f}")
    print(f"  λ₁/λ₂ = {ratio_gt:.2f}")
    
    if ratio_gt < 5:
        print(f"  → GENUINELY 2D ✓")
    elif ratio_gt < 20:
        print(f"  → Anisotropic but still 2D-ish")
    else:
        print(f"  → EFFECTIVELY 1D (very elongated) ✗")
    
    # Predicted
    lam1_pred, lam2_pred, ratio_pred = compute_anisotropy(coords_pred)
    print(f"\nPredicted Coordinates:")
    print(f"  λ₁ = {lam1_pred:.4f},  λ₂ = {lam2_pred:.4f}")
    print(f"  λ₁/λ₂ = {ratio_pred:.2f}")
    
    if ratio_pred < 5:
        print(f"  → GENUINELY 2D ✓")
    elif ratio_pred < 20:
        print(f"  → Anisotropic but still 2D-ish")
    else:
        print(f"  → EFFECTIVELY 1D (very elongated) ✗")
    
    # Anisotropy visualizations
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: Bar comparison
    ax = axes[0, 0]
    methods = ['Ground Truth', 'Predicted']
    ratios = [ratio_gt, ratio_pred]
    colors = ['blue', 'red']
    
    bars = ax.bar(methods, ratios, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    ax.axhline(5, color='g', linestyle='--', linewidth=2, alpha=0.5, label='2D threshold (5)')
    ax.axhline(20, color='orange', linestyle='--', linewidth=2, alpha=0.5, label='1D threshold (20)')
    
    ax.set_ylabel('$\\lambda_1/\\lambda_2$ (Anisotropy Ratio)', fontsize=13, fontweight='bold')
    ax.set_title('Anisotropy Comparison: Ground Truth vs Predicted', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3, axis='y')
    
    for bar, ratio in zip(bars, ratios):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{ratio:.2f}',
                ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    # Plot 2: Eigenvalue scatter
    ax = axes[0, 1]
    ax.scatter(lam2_gt, lam1_gt, c='blue', s=300, marker='o',
              edgecolors='darkblue', linewidth=2, label='Ground Truth', zorder=5)
    ax.scatter(lam2_pred, lam1_pred, c='red', s=300, marker='*',
              edgecolors='darkred', linewidth=2, label='Predicted', zorder=5)
    
    min_val = min(lam2_gt, lam2_pred)
    max_val = max(lam1_gt, lam1_pred)
    ax.plot([min_val, max_val], [min_val, max_val], 'k--',
            linewidth=2, label='$\\lambda_1 = \\lambda_2$', alpha=0.7)
    
    ax.set_xlabel('$\\lambda_2$ (Smaller Eigenvalue)', fontsize=13, fontweight='bold')
    ax.set_ylabel('$\\lambda_1$ (Larger Eigenvalue)', fontsize=13, fontweight='bold')
    ax.set_title('Eigenvalue Comparison', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11, loc='upper left')
    ax.grid(True, alpha=0.3)
    
    # Plot 3: Ground truth coordinates
    ax = axes[1, 0]
    ax.scatter(gt_coords_np[:, 0], gt_coords_np[:, 1], alpha=0.5, s=10,
              c='blue', edgecolors='none')
    ax.set_xlabel('Dimension 1', fontsize=13, fontweight='bold')
    ax.set_ylabel('Dimension 2', fontsize=13, fontweight='bold')
    ax.set_title(f'Ground Truth\n$\\lambda_1/\\lambda_2$ = {ratio_gt:.2f}',
                 fontsize=14, fontweight='bold', color='blue')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal', adjustable='box')
    
    # Plot 4: Predicted coordinates
    ax = axes[1, 1]
    ax.scatter(coords_pred[:, 0], coords_pred[:, 1], alpha=0.5, s=10,
              c='red', edgecolors='none')
    ax.set_xlabel('Dimension 1', fontsize=13, fontweight='bold')
    ax.set_ylabel('Dimension 2', fontsize=13, fontweight='bold')
    ax.set_title(f'Predicted\n$\\lambda_1/\\lambda_2$ = {ratio_pred:.2f}',
                 fontsize=14, fontweight='bold', color='red')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal', adjustable='box')
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*70)
    print("COMPLETE!")
    print("="*70)
else:
    print("\n⚠️ No ground truth coordinates available - skipping metric computation")
    print("   Only showing predicted coordinates...")
    
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.scatter(coords_pred[:, 0], coords_pred[:, 1], s=5, alpha=0.6, c='red')
    ax.set_title('Predicted Coordinates (Slide 3)', fontsize=14, weight='bold')
    ax.set_aspect('equal')
    plt.tight_layout()
    plt.show()


In [ ]:
# ===================================================================
# CELL TYPE VISUALIZATION (GROUND TRUTH vs PREDICTED)
# ===================================================================
print("\n" + "="*70)
print("CELL TYPE VISUALIZATION")
print("="*70)

# Check if stadata3 has level1/level2 celltype columns like scadata
print("\nAvailable columns in stadata3.obs:")
print(list(stadata3.obs.columns))

# If stadata3 has level1_celltype and level2_celltype, apply same logic as scadata
if 'level1_celltype' in stadata3.obs.columns and 'level2_celltype' in stadata3.obs.columns:
    print("\n✓ Found level1_celltype and level2_celltype columns")
    print("   Applying cell type mapping from run_hscc_gems.py...")
    
    stadata3.obs['celltype'] = stadata3.obs['level1_celltype'].astype(str)
    stadata3.obs.loc[stadata3.obs['level1_celltype']=='CLEC9A', 'celltype'] = 'DC'
    stadata3.obs.loc[stadata3.obs['level1_celltype']=='CD1C', 'celltype'] = 'DC'
    stadata3.obs.loc[stadata3.obs['level1_celltype']=='ASDC', 'celltype'] = 'DC'
    stadata3.obs.loc[stadata3.obs['level1_celltype']=='PDC', 'celltype'] = 'PDC'
    stadata3.obs.loc[stadata3.obs['level1_celltype']=='MDSC', 'celltype'] = 'DC'
    stadata3.obs.loc[stadata3.obs['level1_celltype']=='LC', 'celltype'] = 'DC'
    stadata3.obs.loc[stadata3.obs['level1_celltype']=='Mac', 'celltype'] = 'Myeloid cell'
    stadata3.obs.loc[stadata3.obs['level1_celltype']=='Tcell', 'celltype'] = 'T cell'
    stadata3.obs.loc[stadata3.obs['level2_celltype']=='TSK', 'celltype'] = 'TSK'
    stadata3.obs.loc[stadata3.obs['level2_celltype'].isin(['Tumor_KC_Basal', 'Tumor_KC_Diff', 'Tumor_KC_Cyc']), 'celltype'] = 'NonTSK'
    
    cell_type_col = 'celltype'
    cell_types = stadata3.obs[cell_type_col].values
    
    print(f"✓ Cell types created from level1/level2 annotations")

# Otherwise, use simple spatial regions for visualization
else:
    print("\n⚠️  ST data doesn't have cell-level annotations")
    print("   Using spatial regions (grid quadrants) for visualization...")
    
    # Create simple spatial regions based on x,y position
    if 'new_x' in stadata3.obs.columns and 'new_y' in stadata3.obs.columns:
        x_vals = stadata3.obs['new_x'].values
        y_vals = stadata3.obs['new_y'].values
        
        # Divide into 4 quadrants
        x_median = np.median(x_vals)
        y_median = np.median(y_vals)
        
        regions = []
        for x, y in zip(x_vals, y_vals):
            if x < x_median and y < y_median:
                regions.append('Region 1 (Lower-Left)')
            elif x >= x_median and y < y_median:
                regions.append('Region 2 (Lower-Right)')
            elif x < x_median and y >= y_median:
                regions.append('Region 3 (Upper-Left)')
            else:
                regions.append('Region 4 (Upper-Right)')
        
        stadata3.obs['spatial_region'] = regions
        cell_type_col = 'spatial_region'
        cell_types = stadata3.obs[cell_type_col].values
        
        print(f"✓ Created 4 spatial regions based on grid position")
    else:
        print("   Cannot create spatial regions - no coordinates available")
        print("   Skipping cell type visualization")
        cell_types = None

# Only proceed with visualization if we have cell types
if cell_types is not None:
    # Get unique types
    unique_types = np.unique(cell_types)
    n_types = len(unique_types)
    
    print(f"\nFound {n_types} unique categories:")
    for i, ct in enumerate(unique_types):
        count = (cell_types == ct).sum()
        print(f"  {i+1}. {ct}: {count} spots ({count/len(cell_types)*100:.1f}%)")
    
    # Create colormap
    if n_types <= 10:
        cmap = plt.cm.tab10
    elif n_types <= 20:
        cmap = plt.cm.tab20
    else:
        cmap = plt.cm.gist_ncar
    
    type_to_color = {ct: cmap(i / max(n_types, 2)) for i, ct in enumerate(unique_types)}
    
    # ===================================================================
    # CREATE VISUALIZATION
    # ===================================================================
    if gt_coords_norm is not None:
        fig, axes = plt.subplots(1, 2, figsize=(20, 8))
        
        # Plot 1: Ground Truth
        ax = axes[0]
        for ct in unique_types:
            mask = cell_types == ct
            ax.scatter(gt_coords_np[mask, 0], gt_coords_np[mask, 1],
                      c=[type_to_color[ct]], label=ct, s=15, alpha=0.7, edgecolors='none')
        
        ax.set_xlabel('Dimension 1', fontsize=13, fontweight='bold')
        ax.set_ylabel('Dimension 2', fontsize=13, fontweight='bold')
        ax.set_title(f'Ground Truth - {cell_type_col.replace("_", " ").title()}', fontsize=14, fontweight='bold')
        ax.set_aspect('equal', adjustable='box')
        ax.grid(True, alpha=0.3)
        
        # Plot 2: Predicted
        ax = axes[1]
        for ct in unique_types:
            mask = cell_types == ct
            ax.scatter(coords_pred[mask, 0], coords_pred[mask, 1],
                      c=[type_to_color[ct]], label=ct, s=15, alpha=0.7, edgecolors='none')
        
        ax.set_xlabel('Dimension 1', fontsize=13, fontweight='bold')
        ax.set_ylabel('Dimension 2', fontsize=13, fontweight='bold')
        ax.set_title(f'Predicted - {cell_type_col.replace("_", " ").title()}', fontsize=14, fontweight='bold')
        ax.set_aspect('equal', adjustable='box')
        ax.grid(True, alpha=0.3)
        
        # Add legend
        if n_types <= 15:
            handles, labels = axes[1].get_legend_handles_labels()
            fig.legend(handles, labels, loc='center left', bbox_to_anchor=(1.0, 0.5),
                      fontsize=10, title=cell_type_col.replace('_', ' ').title(), 
                      title_fontsize=12, frameon=True)
        
        plt.tight_layout(rect=[0, 0, 0.95, 1])
        plt.show()
    else:
        # Only predicted
        fig, ax = plt.subplots(figsize=(10, 8))
        
        for ct in unique_types:
            mask = cell_types == ct
            ax.scatter(coords_pred[mask, 0], coords_pred[mask, 1],
                      c=[type_to_color[ct]], label=ct, s=15, alpha=0.7, edgecolors='none')
        
        ax.set_xlabel('Dimension 1', fontsize=13, fontweight='bold')
        ax.set_ylabel('Dimension 2', fontsize=13, fontweight='bold')
        ax.set_title(f'Predicted - {cell_type_col.replace("_", " ").title()}', fontsize=14, fontweight='bold')
        ax.set_aspect('equal', adjustable='box')
        ax.grid(True, alpha=0.3)
        
        if n_types <= 15:
            ax.legend(loc='best', fontsize=10, title=cell_type_col.replace('_', ' ').title(), frameon=True)
        
        plt.tight_layout()
        plt.show()

    print("\n" + "="*70)
    print("CELL TYPE VISUALIZATION COMPLETE")
    print("="*70)
else:
    print("\n⚠️  Skipping cell type visualization")


In [ ]:
# ===================================================================
# k-NN PRESERVATION ANALYSIS (ENHANCED)
# ===================================================================
if gt_coords_norm is not None:
    from sklearn.neighbors import NearestNeighbors
    from scipy.stats import spearmanr
    
    print("\n" + "="*70)
    print("k-NN PRESERVATION ANALYSIS (ENHANCED)")
    print("="*70)
    
    # --- Helper functions ---
    def knn_sets(coords, k):
        """Get k-NN indices for all points."""
        nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(coords)
        _, idx = nbrs.kneighbors(coords)
        return idx[:, 1:]
    
    def knn_overlap_frac(idx_gt, idx_pr):
        """Compute per-point overlap fraction: |intersection| / k"""
        n, k = idx_gt.shape
        out = np.empty(n, dtype=np.float32)
        for i in range(n):
            out[i] = len(set(idx_gt[i]).intersection(idx_pr[i])) / k
        return out
    
    def knn_jaccard(idx_gt, idx_pr):
        """Compute per-point Jaccard: |intersection| / |union|"""
        n, k = idx_gt.shape
        out = np.empty(n, dtype=np.float32)
        for i in range(n):
            a = set(idx_gt[i])
            b = set(idx_pr[i])
            out[i] = len(a & b) / max(1, len(a | b))
        return out
    
    def compute_hits(gt_knn_k, pred_knn_m):
        """Compute h_i(k,m) = |G_i^(k) ∩ P_i^(m)| for each point i."""
        n = gt_knn_k.shape[0]
        hits = np.zeros(n, dtype=np.int32)
        for i in range(n):
            gt_set = set(gt_knn_k[i])
            pred_set = set(pred_knn_m[i])
            hits[i] = len(gt_set & pred_set)
        return hits
    
    def recall_at_k(gt_coords, pred_coords, k=10):
        """Recall@k = mean(h_i(k,k) / k)"""
        gt_knn = knn_sets(gt_coords, k)
        pred_knn = knn_sets(pred_coords, k)
        hits = compute_hits(gt_knn, pred_knn)
        return hits.mean() / k, hits / k
    
    def nearmiss_at_m(gt_coords, pred_coords, k_base=10, m=20):
        """NearMiss@m(k) = mean(h_i(k,m) / k)"""
        gt_knn = knn_sets(gt_coords, k_base)
        pred_knn = knn_sets(pred_coords, m)
        hits = compute_hits(gt_knn, pred_knn)
        return hits.mean() / k_base, hits / k_base
    
    def soft_weighted_jaccard(gt_coords, pr_coords, k=20, tau=None):
        """Compute soft distance-weighted Jaccard."""
        nbrs_gt = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(gt_coords)
        d_gt, idx_gt = nbrs_gt.kneighbors(gt_coords)
        d_gt, idx_gt = d_gt[:, 1:], idx_gt[:, 1:]
        
        nbrs_pr = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(pr_coords)
        d_pr, idx_pr = nbrs_pr.kneighbors(pr_coords)
        d_pr, idx_pr = d_pr[:, 1:], idx_pr[:, 1:]
        
        if tau is None:
            tau = np.median(d_gt[:, -1]) + 1e-12
        
        n = gt_coords.shape[0]
        out = np.empty(n, dtype=np.float32)
        
        for i in range(n):
            wgt = {int(j): np.exp(-float(d)/tau) for j, d in zip(idx_gt[i], d_gt[i])}
            wpr = {int(j): np.exp(-float(d)/tau) for j, d in zip(idx_pr[i], d_pr[i])}
            
            keys = set(wgt.keys()) | set(wpr.keys())
            num = 0.0
            den = 0.0
            for j in keys:
                a = wgt.get(j, 0.0)
                b = wpr.get(j, 0.0)
                num += min(a, b)
                den += max(a, b)
            out[i] = num / max(1e-12, den)
        
        return out, tau
    
    # ===================================================================
    # 1) HARD kNN OVERLAP + JACCARD (k=10, 20, 50)
    # ===================================================================
    print("\n--- Hard k-NN Metrics ---")
    
    idx_gt_10 = knn_sets(gt_coords_np, 10)
    idx_pr_10 = knn_sets(coords_pred, 10)
    idx_gt_20 = knn_sets(gt_coords_np, 20)
    idx_pr_20 = knn_sets(coords_pred, 20)
    idx_gt_50 = knn_sets(gt_coords_np, min(50, n_cells-1))
    idx_pr_50 = knn_sets(coords_pred, min(50, n_cells-1))
    
    for k, ig, ip in [(10, idx_gt_10, idx_pr_10), (20, idx_gt_20, idx_pr_20), (min(50, n_cells-1), idx_gt_50, idx_pr_50)]:
        ov = knn_overlap_frac(ig, ip)
        jc = knn_jaccard(ig, ip)
        print(f"[KNN] k={k:2d}: overlap mean={ov.mean():.3f} p50={np.median(ov):.3f} | "
              f"jaccard mean={jc.mean():.3f} p50={np.median(jc):.3f}")
    
    # ===================================================================
    # 2) RECALL@k AND NEARMISS@m METRICS
    # ===================================================================
    print("\n--- Recall and NearMiss Metrics ---")
    
    recall_10, recall_10_per_point = recall_at_k(gt_coords_np, coords_pred, k=10)
    print(f"[RECALL@10] mean={recall_10:.4f} p50={np.median(recall_10_per_point):.4f}")
    
    nearmiss_20, nearmiss_20_per_point = nearmiss_at_m(gt_coords_np, coords_pred, k_base=10, m=20)
    print(f"[NEARMISS@20] (k_base=10) mean={nearmiss_20:.4f} p50={np.median(nearmiss_20_per_point):.4f}")
    
    nearmiss_50, nearmiss_50_per_point = nearmiss_at_m(gt_coords_np, coords_pred, k_base=10, m=min(50, n_cells-1))
    print(f"[NEARMISS@50] (k_base=10) mean={nearmiss_50:.4f} p50={np.median(nearmiss_50_per_point):.4f}")
    
    # ===================================================================
    # 3) SOFT WEIGHTED JACCARD
    # ===================================================================
    print("\n--- Soft Weighted Jaccard ---")
    
    sj20, tau20 = soft_weighted_jaccard(gt_coords_np, coords_pred, k=20, tau=None)
    sj50, tau50 = soft_weighted_jaccard(gt_coords_np, coords_pred, k=min(50, n_cells-1), tau=None)
    print(f"[SOFT-JACCARD] k=20 tau={tau20:.6f}: mean={sj20.mean():.3f} p50={np.median(sj20):.3f}")
    print(f"[SOFT-JACCARD] k=50 tau={tau50:.6f}: mean={sj50.mean():.3f} p50={np.median(sj50):.3f}")
    
    # ===================================================================
    # SUMMARY
    # ===================================================================
    print("\n" + "="*70)
    print("SUMMARY")
    print("="*70)
    
    ov_10 = knn_overlap_frac(idx_gt_10, idx_pr_10).mean()
    jc_10 = knn_jaccard(idx_gt_10, idx_pr_10).mean()
    
    print(f"\n  Global Metrics:")
    print(f"    EDM Pearson:      {pearson_corr:.4f}")
    print(f"    EDM Spearman:     {spearman_corr:.4f}")
    
    print(f"\n  Local Metrics (Hard):")
    print(f"    kNN@10 overlap:   {ov_10:.4f}")
    print(f"    kNN@10 Jaccard:   {jc_10:.4f}")
    print(f"    Recall@10:        {recall_10:.4f}")
    
    print(f"\n  Local Metrics (Tolerant):")
    print(f"    NearMiss@20:      {nearmiss_20:.4f}")
    print(f"    NearMiss@50:      {nearmiss_50:.4f}")
    
    print(f"\n  Local Metrics (Stable):")
    print(f"    Soft Jaccard@20:  {sj20.mean():.4f}")
    
    print("\n" + "="*70)
    print("k-NN ANALYSIS COMPLETE")
    print("="*70)
else:
    print("\n⚠️ No ground truth coordinates available - skipping k-NN preservation analysis")
